In [2]:
import matplotlib.pyplot as plt
import cv2
import PIL
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, Flatten, Dropout, Lambda, InputLayer
from keras.optimizers import Adam
from keras import metrics
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
AllData=[]
with open('/content/drive/MyDrive/trainingdata/data.txt', encoding='utf-8', errors='ignore') as f:
  for line in f.readlines():
    AllData.append(line)

In [4]:
Images=[]
Steering=[]
# Throttle=[]
for i,data in enumerate(AllData):
    position=data.find('.png')
    if position >= 0:
        Images.append('/content/drive/MyDrive/trainingdata/'+data[:(position+4)])
        steering = AllData[i-1].find('Steer:')
        # throttle = AllData[i-1].find('Throttle:')
        firstComma=AllData[i-1].find(',', steering)
        secondComma=AllData[i-1].find(',', firstComma+1)
        Steering.append(float(AllData[i-1][firstComma+2:secondComma]))
        # firstComma=AllData[i-1].find(',', throttle)
        # secondComma=AllData[i-1].find(',', firstComma+1)
        # Throttle.append(float(AllData[i-1][firstComma+2:secondComma]))
print(len(Images))
print(Images[0])

2825
/content/drive/MyDrive/trainingdata/00000194.png


In [5]:
ImagesAsMatrix=[]
for image in Images:
   img=cv2.imread(image)
   img=cv2.resize(img, (215, 120))
   ImagesAsMatrix.append(img)
ImagesAsMatrix=np.asarray(ImagesAsMatrix)

In [6]:
roundedSteering=[]
for i in Steering:
  rounded=round(i,1)
  if rounded==-0.0:
    rounded=0.0
  roundedSteering.append(rounded)
print(roundedSteering)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [7]:
flattened=[]
for i in ImagesAsMatrix:
  flattened.append(i.flatten())

In [9]:
remap={-0.4: 0, -0.3: 1, -0.2: 2, -0.1: 3, 0.0: 4, 0.1:5, 0.2:6, 0.3:7, 0.4:8}
reverser={0:-0.4, 1:-0.3, 2:-0.2, 3:-0.1, 4:0.0, 5:0.1, 6:0.2, 7:0.3, 8:0.4}
remapped=[]
for i in roundedSteering:
  remapped.append(remap[i])
print(remapped)

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 

In [10]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
rus=RandomUnderSampler(sampling_strategy='majority')
ros=RandomOverSampler(sampling_strategy='not majority')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [11]:
X_resampled, y_resampled=rus.fit_resample(np.array(flattened), np.array(remapped))
X_resampled, y_resampled=ros.fit_resample(X_resampled, y_resampled)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [12]:
reshaped=[]
for i in X_resampled:
  reshaped.append(i.reshape(120,215,3))
reshaped=np.asarray(reshaped)

In [13]:
Steerings=[]
for i in y_resampled:
  Steerings.append(reverser[i])

In [15]:
model = Sequential()
model.add(Lambda(lambda x: x/255,input_shape=(120,215,3)))
model.add(Conv2D(24, (5, 5), strides=(2,2), activation='relu'))
model.add(Conv2D(36, (5, 5), strides=(2,2), activation='relu'))
model.add(Conv2D(48, (5, 5), strides=(2,2), activation='relu'))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.summary()
model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.00001), metrics=['mae'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 120, 215, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 58, 106, 24)       1824      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 51, 36)        21636     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 24, 48)        43248     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 20, 64)         76864     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 6, 18, 64)         36928     
_________________________________________________________________
dropout (Dropout)            (None, 6, 18, 64)         0

In [16]:
history=model.fit(reshaped, np.array(Steerings), batch_size=200, epochs=100, validation_split=0.1)

Epoch 1/100
16/16 [==============================] - 63s 3s/step - loss: 0.0583 - mae: 0.2061 - val_loss: 0.1630 - val_mae: 0.4036
Epoch 2/100
16/16 [==============================] - 46s 3s/step - loss: 0.0537 - mae: 0.2008 - val_loss: 0.1579 - val_mae: 0.3973
Epoch 3/100
16/16 [==============================] - 48s 3s/step - loss: 0.0515 - mae: 0.1979 - val_loss: 0.1597 - val_mae: 0.3995
Epoch 4/100
16/16 [==============================] - 46s 3s/step - loss: 0.0476 - mae: 0.1898 - val_loss: 0.1607 - val_mae: 0.4008
Epoch 5/100
16/16 [==============================] - 46s 3s/step - loss: 0.0454 - mae: 0.1868 - val_loss: 0.1476 - val_mae: 0.3841
Epoch 6/100
16/16 [==============================] - 46s 3s/step - loss: 0.0420 - mae: 0.1798 - val_loss: 0.1468 - val_mae: 0.3830
Epoch 7/100
16/16 [==============================] - 47s 3s/step - loss: 0.0381 - mae: 0.1711 - val_loss: 0.1292 - val_mae: 0.3592
Epoch 8/100
16/16 [==============================] - 46s 3s/step - loss: 0.0329 - m

In [17]:
model.save('steeringlambda.h5')

NameError: ignored